# 210707 Upgrade refseq-curated 4815 to d961

In [1]:
import sqlite3
from pathlib import Path
import json

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import alembic.command as cmd

import pandas as pd

In [3]:
from midas.db.migrate import get_alembic_config
from midas.db.models import *
from midas.db.migrate import get_alembic_config

## Setup

In [4]:
TODAY = '2021-07-07'
DATESTR = '210707'

In [5]:
db_dir = Path('/home/jared/projects/midas/data/databases/refseq-curated/2.0')

src_file = db_dir / 'refseq-curated-2.0-r2.db'
dst_file = db_dir / f'refseq-curated-2.0-r3-{DATESTR}.db'

## Read source database into memory

In [6]:
src_connection = sqlite3.connect(str(src_file))

In [7]:
memory_connection = sqlite3.connect(':memory:')

In [8]:
src_connection.backup(memory_connection)

In [9]:
src_connection.close()

## Perform migration in memory

In [10]:
# https://stackoverflow.com/questions/61991050/connect-to-sqlite3-connection-using-sqlalchemy
engine = create_engine('sqlite://', creator = lambda: memory_connection)

In [11]:
config = get_alembic_config(engine)

In [12]:
cmd.current(config)

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


4815cccfb01b


In [13]:
cmd.upgrade(config, 'd961d0698083')

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade 4815cccfb01b -> d961d0698083, Remove unneeded columns


## Fix genome accession nos

In [14]:
engine.execute('UPDATE genomes SET refseq_acc = genbank_acc;')

In [15]:
engine.execute('UPDATE genomes SET genbank_acc = NULL;')

## Update metadata

In [16]:
dict(next(engine.execute('SELECT * FROM reference_genome_sets LIMIT 1')))

{'key': 'midas/refseq-bacterial-assemblies/curated',
 'version': '2.0',
 'id': 3,
 'name': 'MIDAS curated RefSeq bacterial WGS assemblies',
 'description': 'Curated subset of RefSeq bacterial genomes in NCBI assembly database',
 'extra': None}

In [17]:
revision_data = dict(
    num=3,
    date_created=TODAY,
    description='Updated to alembic revision d961d0698083. Removed Entrez ESummary data. Fixed refseq_acc values incorrectly stored in genbank_acc column.',
)

In [18]:
extra = dict(revision=revision_data)
extra_json = json.dumps(extra)

engine.execute('UPDATE reference_genome_sets SET extra = :e', dict(e=extra_json))

## Write output to file

In [19]:
if dst_file.is_file():
    dst_file.unlink()

In [20]:
memory_connection.execute(f'VACUUM INTO "{dst_file}"')